<a href="https://colab.research.google.com/github/joseorlandomx/topicodeindustria1/blob/main/Practica_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica 3**

José Orlando Salas Contreras (208743738)

[jose.salas4373@alumnos.udg.mx](mailto:jose.salas4373@alumnos.udg.mx)

**Requisitos mínimos:**
* **Actividad 1:** Path-length - (BM1 vs BM2 vs CRW) (4 pts)
* **Actividad 2:** Mean Squared Displacement - (BM vs CRW) (4 pts)
* **Actividad 3:** Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)
* **Actividad 4:** Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)

**Revisión de la práctica:**
* El notebook con la práctica se entregará en un repositorio en **GitHub**. Será necesario demostrar por medio de commits el historial de versiones de su NoteBook.
* El estudiante deberá ser capaz de explicar su código y de corregir errores introducidos a este.

## Modules

In [225]:
import math
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance

## Classes

In [61]:
# Nota: Esta clase la importaremos junto con el segundo bloque de modulos
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## **Actividad 1: Path-length - (BM1 vs BM2 vs CRW) (4 pts)**
* Implementar función que genere **Brownian Motions (BM)** utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks (CRW)** utilizando **pandas**.
* Implementar una **función** alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

### **Actividad 1** con Brownian Motions y Correlated Random Walks generados con funciones

In [147]:
# Función para generar Brownian Motions (BM) con Pandas
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
    n_steps:
    speed:
    s_pos:
  Returns:
    BM_2d_df:
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

In [148]:
# Función para generar Correlated Random Walks (CRW) con pandas
def generate_crw(n_steps, alpha, beta, start=(0, 0)):
    """
    Genera Correlated Random Walks (CRW) utilizando pandas.

    Parameters:
    - n_steps (int): Número de pasos en la caminata.
    - alpha (float): Parámetro de escala para la distribución normal que afecta los cambios en x.
    - beta (float): Parámetro de escala para la distribución normal que afecta los cambios en y.
    - start (tuple): Coordenadas iniciales (por defecto, (0, 0)).

    Returns:
    - pandas.DataFrame: DataFrame con las coordenadas x e y de la caminata.
    """
    np.random.seed(42)  # Fijar semilla para reproducibilidad

    # Inicializar el DataFrame
    crw_df = pd.DataFrame(index=range(n_steps), columns=['x_pos', 'y_pos'])

    # Establecer las coordenadas iniciales
    crw_df.loc[0] = start

    # Generar la CRW
    for i in range(1, n_steps):
        dx = np.random.normal(0, alpha)
        dy = beta * crw_df['x_pos'][i - 1] + np.random.normal(0, 1)

        crw_df.loc[i, 'x_pos'] = crw_df['x_pos'][i - 1] + dx
        crw_df.loc[i, 'y_pos'] = crw_df['y_pos'][i - 1] + dy

    return crw_df

In [149]:
# BM con vel 3
BM_2d_df_3 = bm_2d(1000, 3, [0,0])

# BM con vel 6
BM_2d_df_6 = bm_2d(1000, 6, [0,0])

# CRW con alpha 0.1 y beta 0.3
CRW_2d_df_9 = generate_crw(1000, 0.1, 0.3, [0,0])

In [ ]:
# Calcular distancia entre detecciones consecutivos
dis_BM_3 = np.array([distance.euclidean(BM_2d_df_3.iloc[i-1],BM_2d_df_3.iloc[i]) for i in range(1,BM_2d_df_3.shape[0])])
# Calculo del path length
pl_BM_3 = np.cumsum(dis_BM_3)

# Calcular distancia entre detecciones consecutivas
dis_BM_6 = np.array([distance.euclidean(BM_2d_df_6.iloc[i-1],BM_2d_df_6.iloc[i]) for i in range(1,BM_2d_df_6.shape[0])])
# Calculo del path length
pl_BM_6 = np.cumsum(dis_BM_6)

# Calcular distancia entre detecciones consecutivas
dis_CRW_6 = np.array([distance.euclidean(CRW_2d_df_9.iloc[i-1],CRW_2d_df_9.iloc[i]) for i in range(1,CRW_2d_df_9.shape[0])])
# Calculo del path length
pl_CRW_6 = np.cumsum(dis_CRW_6)

In [ ]:
# Inicializamos la gráfica
fig_path_length = go.Figure()

# Trazamos el Brownian Motion con velocidad 3 (BM_3)
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_3))+1,
    y = pl_BM_3,
    line = dict(width = 2, color = 'green'),
    name = 'path_length_BM_3',
    showlegend = True
))

# Trazamos el Brownian Motion con velocidad 6 (BM_6)
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_6))+1,
    y = pl_BM_6,
    line = dict(width = 6, color = 'red'),
    name = 'path_length_BM_6',
    showlegend = True
))

# Trazamos el Correlated Random Walks (CRW6)
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_CRW_6))+1,
    y = pl_CRW_6,
    line = dict(width = 2, color = 'yellow'),
    name = 'path_length_CRW_6',
    showlegend = True
))

# Personaliamos las títulos de los ejes de la gráfica
fig_path_length.update_layout(title='Brownian Motion and CRW - Path Length',
                              xaxis_title='Step',
                              yaxis_title='Normalized Length')  # Actualizado el título del eje y

# Mostramos la gráfica
fig_path_length.show()

### **Actividad 1** con Brownian Motions y Correlated Random Walks cargados desde archivos CSV

In [137]:
# Función para convertir valores del DataFrame de string a numéricos
def convert_to_numeric(trajectory):
    # Convertir las columnas 'x_pos' e 'y_pos' a numérico
    trajectory['x_pos'] = pd.to_numeric(trajectory['x_pos'], errors='coerce')
    trajectory['y_pos'] = pd.to_numeric(trajectory['y_pos'], errors='coerce')
    return trajectory

# Función para obtener los valores del path length (distancia y longitud)
def get_path_length(trajectory):
    #trajectory = convert_to_numeric(trajectory)
    distance = [np.linalg.norm(trajectory.iloc[i-1] - trajectory.iloc[i]) for i in range(1, trajectory.shape[0])]
    length = np.cumsum(distance)
    return distance, length

In [134]:
# Cargamos las trayectorias desde los archivos CSV y obtenemos sus path lengths con la función

# Trayectoria Brownian Motion con velocidad 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')
PL_BM_3 = get_path_length(BM_2d_df_3)

# Trayectoria Brownian Motion con velocidad 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')
PL_BM_6 = get_path_length(BM_2d_df_6)

# Trayectoria Correlated Random Walks
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')
PL_CRW_6 = get_path_length(CRW_2d_df_9)

In [ ]:
# Normalizamos las longitudes acumulativas para hacer que todas las líneas sean visibles
max_length = max(max(PL_BM_3[1]), max(PL_BM_6[1]), max(PL_CRW_6[1]))

# Inicializamos la gráfica
fig_path_length = go.Figure()

# Trazamos el Brownian Motion con velocidad 3 (BM_3)
fig_path_length.add_trace(go.Scatter(x=np.arange(1, len(PL_BM_3[1]) + 1), y=PL_BM_3[1]/max_length,
                                     mode='lines',
                                     name='Path Length BM3', line = dict(width = 2, color = 'green'),
                                     showlegend = True))

# Trazamos el Brownian Motion con velocidad 6 (BM_6)
fig_path_length.add_trace(go.Scatter(x=np.arange(1, len(PL_BM_6[1]) + 1), y=PL_BM_6[1]/max_length,
                                     mode='lines',
                                     name='Path Length BM6', line = dict(width = 6, color = 'red'),
                                     showlegend = True))

# Trazamos el Correlated Random Walks (CRW6)
fig_path_length.add_trace(go.Scatter(x=np.arange(1, len(PL_CRW_6[1]) + 1), y=PL_CRW_6[1]/max_length,
                                     mode='lines',
                                     name='Path Length CRW6', line=dict(width = 2, color = 'yellow'),
                                     showlegend = True))

# Personaliamos las títulos de los ejes de la gráfica
fig_path_length.update_layout(title='Brownian Motion and CRW - Path Length',
                              xaxis_title='Step',
                              yaxis_title='Normalized Length')  # Actualizado el título del eje y

# Mostramos la gráfica
fig_path_length.show()

## **Actividad 2: Mean Squared Displacement - (BM vs CRW) (4 pts)**
* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en **Pandas** Data Frame.
* Visualizar con **plotly**.

In [217]:
# Trayectoria Brownian Motion (BM)
BM_trajectory = pd.read_csv('trajectories/brownian_3.csv')

# Trayectoria Correlated Random Walks (CRW)
CRW_trajectory = pd.read_csv('trajectories/crw_6_9.csv')

In [218]:
# Función para calcular MSD (Mean Squared Displacement)
def calculate_msd(trajectory):
    # Lista para almacenar los valores de MSD
    msd_values = []

    # Iterar desde tau=1 hasta el tamaño total de la trayectoria
    for tau in range(1, trajectory.shape[0]):

        # Calcular los desplazamientos en las coordenadas x e y
        displacement_x = trajectory['x_pos'].values[tau:] - trajectory['x_pos'].values[:-tau]
        displacement_y = trajectory['y_pos'].values[tau:] - trajectory['y_pos'].values[:-tau]

        # Calcular el cuadrado del desplazamiento
        squared_displacement = displacement_x**2 + displacement_y**2

        # Calcular el valor medio del cuadrado del desplazamiento
        msd = np.mean(squared_displacement)

        # Agregar el valor de MSD a la lista
        msd_values.append(msd)

    # Devolver la lista de valores de MSD
    return msd_values

In [229]:
# Calcular los valores de la curva de mean squared displacement
MSD_BM = calculate_msd(BM_trajectory)
MSD_CRW = calculate_msd(CRW_trajectory)

# Guardar métricas en Pandas Data Frame
msd_metrics = pd.DataFrame({
    'TAU': range(1, len(MSD_BM) + 1),
    'MSD_BM': MSD_BM,
    'MSD_CRW': MSD_CRW
})

# Exportar métricas a un archivo CSV
msd_metrics.to_csv('trajectories/actividad2_metrics.csv', index=False)

# Leer el archivo CSV exportado
msd_df = pd.read_csv('trajectories/actividad2_metrics.csv')

# Eliminamos el archivo temporal
os.remove("trajectories/actividad2_metrics.csv")

In [ ]:
# Inicializamos la gráfica
fig = go.Figure()

# Añadimos una traza (trace) para MSD de Brownian Motion (BM)
fig.add_trace(go.Scatter(
    x=msd_df['TAU'],            # Eje X: Valores de tau
    y=msd_df['MSD_BM'],          # Eje Y: Valores de MSD para BM
    mode='lines',               # Modo de la traza: líneas
    name='MSD BM'                # Etiqueta de la traza
))

# Añadimos otra traza para MSD de Correlated Random Walks (CRW)
fig.add_trace(go.Scatter(
    x=msd_df['TAU'],            # Eje X: Valores de tau
    y=msd_df['MSD_CRW'],         # Eje Y: Valores de MSD para CRW
    mode='lines',               # Modo de la traza: líneas
    name='MSD CRW'               # Etiqueta de la traza
))

# Personalizamos los títulos de los ejes de la gráfica
fig.update_layout(
    title='Mean Squared Displacement (Brownian Motion vs Correlated Random Walks)',  # Título de la gráfica
    xaxis_title='TAU',           # Título del eje X
    yaxis_title='Mean Squared Displacement (MSD)'  # Título del eje Y
)

# Mostramos la gráfica
fig.show()

## **Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)**
* Generar dos **CRWs** con dos exponentes diferentes.
* Guardar trayectorias en **pandas** Data Frame.
* Implementar una **función** que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con **plotly**.

In [270]:
# Obtenemos dos Correlated Random Walks a partir de archivos
CRW_1 = pd.read_csv('trajectories/crw_6_6.csv')
CRW_2 = pd.read_csv('trajectories/crw_6_9.csv')

In [271]:
# Función para calcular los valores de turning angle de una trayectoria
def calculate_turning_angle(trajectory):
    """
    Calcula los valores de turning angle para una trayectoria.

    Parameters:
    - trajectory (pandas.DataFrame): DataFrame con las coordenadas de la trayectoria.

    Returns:
    - numpy.ndarray: Array con los valores de turning angle.
    """
    # Calculamos las diferencias en las coordenadas
    dx = np.diff(trajectory['x_pos'])
    dy = np.diff(trajectory['y_pos'])

    # Calculamos el ángulo utilizando la arctangente y convertimos a grados
    turning_angle = np.degrees(np.arctan2(dy, dx))

    return turning_angle

In [272]:
# Calculamos los valores de turning angle para cada CRW
turning_angle_1 = calculate_turning_angle(CRW_1)
turning_angle_2 = calculate_turning_angle(CRW_2)

In [ ]:
# Crear histogramas para cada CRW
hist_1 = np.histogram(turning_angle_1, bins=50, density=True)
hist_2 = np.histogram(turning_angle_2, bins=50, density=True)

# Crear gráfico con Plotly
fig = go.Figure()



h1 = go.Histogram(x=hist_1[1], y=hist_1[0], opacity=0.75, name='Observed=0.6')
h2 = go.Histogram(x=hist_2[1], y=hist_2[0], opacity=0.75, name='Observed=0.9')
l1 = go.Scatter(
    x=CRW_1['x_pos'],            # Eje X: Valores de tau
    y=CRW_1['y_pos'],         # Eje Y: Valores de MSD para CRW
    mode='lines',               # Modo de la traza: líneas
    name='CRW_6'               # Etiqueta de la traza
)
l2 = go.Scatter(
    x=CRW_2['x_pos'],            # Eje X: Valores de tau
    y=CRW_2['y_pos'],         # Eje Y: Valores de MSD para CRW
    mode='lines',               # Modo de la traza: líneas
    name='CRW_9'               # Etiqueta de la traza
)

fig = go.Figure(data=[h1, h2, l1, l2])

# Personalizar diseño
fig.update_layout(title='Turning-angle Distribution for CRWs with Different Alphas',
                  xaxis_title='Turning Angle (degrees)',
                  yaxis_title='Density',
                  barmode='overlay')

# Mostrar la figura
fig.show()

In [ ]:
# Generar datos de ejemplo
np.random.seed(42)
data = np.random.normal(0, 1, 1000)

# Crear histograma
histogram = go.Histogram(x=data, nbinsx=30, opacity=0.7, name='Histogram')

# Calcular la función de densidad de probabilidad (PDF)
pdf_values, bin_edges = np.histogram(data, bins=30, density=True)
pdf_line = go.Scatter(x=(bin_edges[:-1] + bin_edges[1:]) / 2, y=pdf_values, mode='lines', name='PDF')

# Crear gráfico con Plotly
fig = go.Figure(data=[histogram, pdf_line])

# Personalizar diseño
fig.update_layout(title='Histogram with Probability Density Function (PDF)',
                  xaxis_title='Value',
                  yaxis_title='Frequency/Density')

# Mostrar la figura
fig.show()

## **Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)**
* Implementar función que genere **Lévy Walks** (LW) utilizando **pandas**.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una **función** que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener Step-length distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con **plotly**.

In [279]:
def generate_levy_walk(n_steps, alpha, scale):
    """
    Genera una Lévy Walk utilizando pandas.

    Parameters:
    - n_steps (int): Número de pasos en la trayectoria.
    - alpha (float): Parámetro alpha que controla la distribución de Lévy.
    - scale (float): Escala para ajustar la longitud de los pasos.

    Returns:
    - pandas.DataFrame: DataFrame con las coordenadas de la trayectoria.
    """
    # Inicializar DataFrame
    trajectory_df = pd.DataFrame(columns=['x_pos', 'y_pos'])

    # Inicializar posición
    x_pos, y_pos = 0, 0

    for step in range(n_steps):
        # Generar longitud del paso según distribución de Lévy
        step_length = np.random.pareto(alpha) * scale

        # Generar ángulo aleatorio
        angle = np.random.uniform(0, 2 * np.pi)

        # Calcular nueva posición
        x_pos += step_length * np.cos(angle)
        y_pos += step_length * np.sin(angle)

        # Agregar posición al DataFrame
        trajectory_df = pd.concat([trajectory_df, pd.DataFrame({'x_pos': x_pos, 'y_pos': y_pos}, index=[0])],
                                  ignore_index=True)

    return trajectory_df

In [280]:
def calculate_step_length(trajectory):
    """
    Calcula los valores de step length para una trayectoria.

    Parameters:
    - trajectory (pandas.DataFrame): DataFrame con las coordenadas de la trayectoria.

    Returns:
    - numpy.ndarray: Array con los valores de step length.
    """
    differences = np.diff(trajectory[['x_pos', 'y_pos']], axis=0)
    step_lengths = np.linalg.norm(differences, axis=1)
    return step_lengths

In [281]:
# Generar Lévy Walks
levy_walk_1 = generate_levy_walk(n_steps=500, alpha=1.5, scale=1.0)
levy_walk_2 = generate_levy_walk(n_steps=500, alpha=2.0, scale=1.0)

# Guardar trayectorias en Pandas Data Frame
levy_walk_1.to_csv('levy_walk_1.csv', index=False)
levy_walk_2.to_csv('levy_walk_2.csv', index=False)

# Calcular valores de step length
step_lengths_1 = calculate_step_length(levy_walk_1)
step_lengths_2 = calculate_step_length(levy_walk_2)

In [282]:
# Crear histogramas para cada Lévy Walk
hist_1 = np.histogram(step_lengths_1, bins=50, density=True)
hist_2 = np.histogram(step_lengths_2, bins=50, density=True)

In [283]:
# Crear gráfico con Plotly
fig = go.Figure()

# Agregar histograma para Lévy Walk con alpha=1.5
fig.add_trace(go.Histogram(x=hist_1[1], y=hist_1[0], opacity=0.75, name='Alpha=1.5'))

# Agregar histograma para Lévy Walk con alpha=2.0
fig.add_trace(go.Histogram(x=hist_2[1], y=hist_2[0], opacity=0.75, name='Alpha=2.0'))

# Personalizar diseño
fig.update_layout(title='Step-length Distribution for Lévy Walks with Different Alphas',
                  xaxis_title='Step Length',
                  yaxis_title='Density',
                  barmode='overlay')

# Mostrar la figura
fig.show()